# Loading datasettrain = pd.read_csv('../input/herbarium-2022-pandas/train.csv')



In [1]:
import os
import gc
import numpy as np
import pandas as pd
from PIL import Image
import cv2 as cv
from matplotlib import pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import train_test_split

In [2]:
from torch.utils.data import DataLoader, Dataset
import torch
from torchvision import transforms
from PIL import Image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import time
import os
import copy

In [3]:
device

In [4]:
# train = pd.read_json(path_or_buf = 'file:///kaggle/input/herbarium-2022-fgvc9/train_metadata.json', orient='recordstrain_dir = '../input/herbarium-2022-fgvc9/train_images/'
train_dir = '../input/herbarium-2022-fgvc9/train_images/'

with open("../input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)

image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

train_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
train_df['genus'] = train_df['genus'].map(genus_map)

train_df.head().style.set_properties(**{'background-color': 'black',
                           'color': 'white',
                           'border-color': 'white'})


In [5]:
cat_val_cnt = train_df['category'].value_counts()
cat_val_cnt

We have too many categories, so let's reduce them.

In [6]:
cat_index = cat_val_cnt[cat_val_cnt == 80].sort_values(ascending=False).index
# cat_index = cat_index[:20]

In [7]:
herb_train_df = train_df[train_df.category.isin(cat_index)]
herb_train_df.info()

In [8]:
herb_train_df['category'].value_counts()

In [9]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
herb_train_df['category'] = label_encoder.fit_transform(herb_train_df['category'])

In [10]:
herb_train_df['category']

In [11]:
X_train, X_val, y_train, y_val = train_test_split(herb_train_df['image_dir'].to_numpy(), herb_train_df['category'].to_numpy(), test_size=0.25, random_state=42)

In [12]:
y_train

In [26]:
# Number of classes in the dataset
num_classes = 174

# Batch size for training (change depending on how much memory you have)
batch_size = 600

# Number of epochs to train for
num_epochs = 60

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

#Flag to use pretrained model
use_pretrained = True


In [14]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


In [15]:
""" ResNeXt101
        """
# model_ft = models.resnext101_32x8d(pretrained=use_pretrained)
model_ft = models.resnet18(pretrained=use_pretrained)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, num_classes)
input_size = 224

In [17]:
print(model_ft)

In [18]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [19]:
class Dataset(Dataset):
    def __init__(self, x, labels, val_train, transform=None):
        self.val_train = val_train
        self.file_names = x
        self.labels = labels
        self.transform = transform[val_train]
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = Image.open(file_path)
        image = image.convert("RGB")        
        if self.transform:
            image = self.transform(image)
#         image = np.array(image)
#         image = image.unsqueeze(0)
        label = torch.tensor(self.labels[idx]).long()
        return image, label


In [20]:
train_dataset = Dataset(X_train, y_train, val_train = 'train', transform=data_transforms)
val_dataset = Dataset(X_val, y_val, val_train = 'val', transform=data_transforms)

for i in range(5):
    plt.figure(figsize=(4, 4))
    image, label = train_dataset[i]
    plt.imshow(image[0])
    plt.title(f'label: {label}')
    plt.show() 

In [27]:
# dataloaders_dict = {'train' : train_dataset,
#                    'val' : val_dataset}
dataloaders_dict = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1),
                   'val' : torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=1)}

In [22]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
#                 print('Device: ', device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [28]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

In [29]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)


In [31]:
ohist = []

ohist = [h.cpu().numpy() for h in hist]
plt.figure(figsize=(15, 10))
plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
plt.ylim((0,1.0))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

In [ ]:
test_dir = '../input/herbarium-2022-fgvc9/test_images/'
with open("../input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    train_meta = json.load(json_file)

image_ids = [image["image_id"] for image in train_meta]
image_dirs = [test_dir + image['file_name'] for image in train_meta]

test_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs})

test_df.head().style.set_properties(**{'background-color': 'black',
                           'color': 'white',
                           'border-color': 'white'})


In [ ]:
test_x = test_df["image_dir"].to_numpy()

In [ ]:
class TestDataset(Dataset):
    def __init__(self, x, transform=None):
        self.file_names = x
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        image = Image.open(file_path)
        image = image.convert("RGB")        
        if self.transform:
            image = self.transform['val'](image)
        return image.unsqueeze(0)


In [ ]:
test_dataset = TestDataset(test_x, data_transforms)

In [ ]:
model_ft.eval()
Y_pred = [model_ft(inp.to(device)) for inp in test_dataset]

In [ ]:
Y_pred